In [1]:
import pandas as pd
sleep_disorder = pd.read_csv('./data/Sleep_health_and_lifestyle_dataset.csv')
sleep_disorder.head(3)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN


In [4]:
sleep_disorder.loc[sleep_disorder['Sleep Disorder'].isna(), 'Sleep Disorder'] = 'no'
sleep_disorder.head(3)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,no
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,no
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,no


In [9]:
sleep_disorder[['Systolic_pressure', 'Diastolic_pressure']] = (sleep_disorder['Blood Pressure'].str.split('/', expand=True)).astype(int)
sleep_disorder.head(3)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder,Systolic_pressure,Diastolic_pressure
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,no,126,83
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,no,125,80
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,no,125,80


In [11]:
sleep_disorder[['Systolic_pressure', 'Diastolic_pressure']].describe()

,Systolic_pressure,Diastolic_pressure
count,374.000000,374.000000
mean,128.553476,84.649733
std,7.748118,6.161611
min,115.000000,75.000000
25%,125.000000,80.000000
50%,130.000000,85.000000
75%,135.000000,90.000000
max,142.000000,95.000000


In [12]:
sleep_disorder['hypertension_category'] = 0
sleep_disorder.loc[(sleep_disorder['Systolic_pressure'] >= 120) & (sleep_disorder['Systolic_pressure'] <= 129) & (sleep_disorder['Diastolic_pressure'] <= 80), 'hypertension_category'] = 1
sleep_disorder.loc[((sleep_disorder['Systolic_pressure'] >= 130) & (sleep_disorder['Systolic_pressure'] <= 139)) | ((sleep_disorder['Diastolic_pressure'] > 80) & (sleep_disorder['Diastolic_pressure'] <= 89)), 'hypertension_category'] = 2
sleep_disorder.loc[(sleep_disorder['Systolic_pressure'] >= 140) | (sleep_disorder['Diastolic_pressure'] >= 90), 'hypertension_category'] = 3

sleep_disorder['hypertension_category'].describe()

count    374.000000
mean       1.748663
std        0.971958
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        3.000000
Name: hypertension_category, dtype: float64

In [23]:
sleep_disorder['Occupation'], occupation_map = pd.factorize(sleep_disorder['Occupation'])
sleep_disorder['BMI Category'], bmi_map = pd.factorize(sleep_disorder['BMI Category'])
sleep_disorder['Gender'], gender_map = pd.factorize(sleep_disorder['Gender'])

sleep_disorder.head(3)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder,Systolic_pressure,Diastolic_pressure,hypertension_category
0,1,0,27,0,6.1,6,42,6,0,126/83,77,4200,no,126,83,2
1,2,0,28,1,6.2,6,60,8,1,125/80,75,10000,no,125,80,1
2,3,0,28,1,6.2,6,60,8,1,125/80,75,10000,no,125,80,1


In [24]:
sleep_disorder['Sleep Disorder'], sleep_disorder_map = pd.factorize(sleep_disorder['Sleep Disorder'])

In [25]:
sleep_disorder['Heart Rate'].describe()

count    374.000000
mean      70.165775
std        4.135676
min       65.000000
25%       68.000000
50%       70.000000
75%       72.000000
max       86.000000
Name: Heart Rate, dtype: float64

In [26]:
sleep_disorder_to_train = sleep_disorder.drop(columns=['Person ID', 'Systolic_pressure', 'Diastolic_pressure', 'Blood Pressure'])

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sleep_disorder_to_train.drop(columns=['Sleep Disorder']), sleep_disorder_to_train['Sleep Disorder'])
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test)

In [34]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=X_test.shape[1:]),
    tf.keras.layers.Dense(units=5,activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(3,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)

model.fit(X_train, y_train, epochs=10, callbacks=[early_stop])
model.evaluate(X_test,  y_test, verbose=2)
#There are not sufficient data to train the model

Epoch 1/10
9/9 [==============================] - 1s 5ms/step - loss: 2918.9111 - accuracy: 0.2357
Epoch 2/10
9/9 [==============================] - 0s 4ms/step - loss: 2852.4897 - accuracy: 0.2429
Epoch 3/10
9/9 [==============================] - 0s 6ms/step - loss: 2720.0244 - accuracy: 0.2214
Epoch 4/10
9/9 [==============================] - 0s 6ms/step - loss: 2453.5439 - accuracy: 0.2286
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 2342.1453 - accuracy: 0.2500
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 2221.0200 - accuracy: 0.2000
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 2103.2363 - accuracy: 0.2286
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 1723.2390 - accuracy: 0.3286
Epoch 9/10
9/9 [==============================] - 0s 4ms/step - loss: 1592.9094 - accuracy: 0.3357
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 1550.1888 - accuracy: 0.3143
3/3 - 0s 

[1168.8077392578125, 0.2142857164144516]